In [0]:
# Load libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from statsmodels.formula.api import ols
import urllib

In [0]:
# Load data and rename columns
link = "https://github.com/chiaracecilia/forecasting_competition/raw/master/load_weather_dataset.xlsx"
urllib.request.urlretrieve(link, "load_weather_dataset_recent.xlsx")
data = pd.read_excel('load_weather_dataset_recent.xlsx')

data.columns=['Date','Temp_min','Temp_mean','Temp_max','Load','London_min','London_mean','London_max',
              'Bristol_min','Bristol_mean','Bristol_max','Leeds_min','Leeds_mean','Leeds_max','Lockdown_index']

# Last date is missing, clean up
data.iloc[-1,0]='2020-05-31'

In [0]:
# Convert to datetime and add day of the week, weekend flag, lockdown and convert Load to log load (base e)
data['Date'] = pd.to_datetime(data['Date'])
data['Day'] = data['Date'].apply(lambda x: x.weekday())
data['Month'] = data['Date'].apply(lambda x: x.month)
data['Weekend'] = data['Day'].apply(lambda x: 1 if x>=5 else 0)
lockdown_date = dt.datetime(2020,3,20) # This is the date we will assume for lockdown
data['Lockdown_index'] = data['Lockdown_index'].apply(lambda x: 0 if pd.isnull(x) else x)
data['Load'] = np.log(data['Load'])
data['t'] = (data['Date']-data['Date'][0]) 
data['t'] = data['t'].apply(lambda x: pd.Timedelta.total_seconds(x) / (24*60*60*365)) # Compute t in year fraction
data['t_2'] = data['t']**2 # Compute t^2

## Added forecast for N to enable forecast for N+1 (here I have put the forecast for 27 May, but we need forecast for 28 May ultimately)
#data.loc[data['Date']==dt.datetime(2020,5,27),['Load']]=9.889952197763487
#data.loc[data['Date']==dt.datetime(2020,5,27),['Temp_mean']]=(18.5 + 18.5 + 15)/3

In [0]:
# Add Bank holiday flags, as energy demand is lower on bank holiday days
bh_ls = ['2017-01-01', '2017-04-14', '2017-05-01', '2017-05-29', '2017-12-25', '2017-12-26',
         '2018-01-01', '2018-03-30','2018-05-07', '2018-12-28', '2018-12-25', '2018-12-26',
         '2019-01-01', '2019-04-19', '2019-04-22', '2019-05-06', '2019-05-27', '2019-12-24', '2019-12-25', '2019-12-26',
         '2020-01-01', '2020-04-10', '2020-04-13','2020-04-20','2020-04-21','2020-04-22','2020-05-08', '2020-05-25']

bd_dt = [pd.to_datetime(x) for x in bh_ls]
data['Holiday'] = data['Date'].apply(lambda x: 1 if x in bd_dt else 0)

In [324]:
data.tail()

Date  Temp_min  Temp_mean  ...         t        t_2  Holiday
1242 2020-05-27       8.9       15.9  ...  3.402740  11.578638        0
1243 2020-05-28      10.4       17.0  ...  3.405479  11.597290        0
1244 2020-05-29      10.1       17.1  ...  3.408219  11.615958        0
1245 2020-05-30       NaN        NaN  ...  3.410959  11.634641        0
1246 2020-05-31       NaN        NaN  ...  3.413699  11.653338        0

[5 rows x 21 columns]

In [0]:
# Add smoothed series
data['Sma'] = data['Load'].rolling(window=7).mean() # Weekly moving average
data['Ema_10'] = data['Load'].ewm(alpha=0.1).mean() # Exponential smoothing with alpha = 0.1
data['Ema_50'] = data['Load'].ewm(alpha=0.5).mean() # Exponential smoothing with alpha = 0.5
data['Ema_90'] = data['Load'].ewm(alpha=0.9).mean() # Exponential smoothing with alpha = 0.9
data['Ema_95'] = data['Load'].ewm(alpha=0.99).mean() # Exponential smoothing with alpha = 0.95
# Add lags for regression, e.g. Autoregressive(1)
data['Ema_90_1'] = data['Ema_90'].shift(periods = 1)
data['Ema_95_1'] = data['Ema_95'].shift(periods = 1)

# Add a moving average for temperature
data['Temp_mean_ma'] = data['Temp_mean'].rolling(window=7).mean()

In [326]:
# Add seasonal factors (one factor for each day of the week), pre-lockdown
avg = data[data['Date']<lockdown_date]['Ema_90'].mean()

i_factors = data[data['Date']<dt.datetime(2020,3,20)][['Ema_90','Day']].groupby(['Day']).mean().reset_index()
i_factors.rename(columns={'Ema_90':'Avg_Ema_90'},inplace=True)
i_factors['i_t'] = i_factors['Avg_Ema_90'] / avg

# Map this back to the df
data = data.merge(i_factors[['Day','i_t']], on='Day',how='left')
data.head()

Date  Temp_min  Temp_mean  ...   Ema_95_1  Temp_mean_ma       i_t
0 2017-01-01       4.4        5.2  ...        NaN           NaN  0.989582
1 2017-01-02      -0.6        2.0  ...  10.206550           NaN  1.001178
2 2017-01-03      -1.2        2.7  ...  10.293580           NaN  1.004336
3 2017-01-04       2.2        4.9  ...  10.435634           NaN  1.004729
4 2017-01-05      -3.2        1.2  ...  10.452794           NaN  1.004354

[5 rows x 30 columns]

In [327]:
# Create line graph
cutoff = dt.datetime(2000,1,1)
plt_df = data[data['Date']>=cutoff]
fig = go.Figure()
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Load'],
                    mode='lines',
                    name='Log load'))
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Ema_10'],
                    mode='lines',
                    name='Smoothed Exp 0.1'))
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Ema_90'],
                    mode='lines',
                    name='Smoothed Exp 0.9'))

fig.update_layout(
    title="Log load history",
    xaxis_title="Time",
    yaxis_title="Log load"
)
fig.show()

In [328]:
# Scatter of temperature mean and load
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Temp_mean'], y=data['Ema_90'],
                    mode='markers',
                    text=data['Date'],
                    marker_color=data['Weekend'],
                    name='Log load'))

fig.update_layout(
    title="Load vs mean temperature (weekends in yellow)",
    xaxis_title="Mean temperature",
    yaxis_title="Smoothed log load (exp 0.9)"
)

fig.show()

In [0]:
# If we observe the plot of load vs. temperature, it is non-linear: how to incorporate this effect?
# Naive approach of splitting into two, by inspection:
# Choose 2 knots: T <= 15.5 (linear), T>15.5 (linear)
# We assume it is linear below 15.5 and linear above 15.5 (with different slopes). Since we will be forecasting in late spring / early summer only
# This assumption is probably fine

data['T_mean_15_lt'] = data['Temp_mean'].apply(lambda x: x if x<=15.5 else 0)
data['T_mean_15_bt'] = data['Temp_mean'].apply(lambda x: x if x>15.5 else 0)

# Do the same for the temperature moving average
data['T_mean_ma_15_lt'] = data['Temp_mean_ma'].apply(lambda x: x if x<=15.5 else 0)
data['T_mean_ma_15_bt'] = data['Temp_mean_ma'].apply(lambda x: x if x>15.5 else 0)


In [330]:
# Try basic regression
# Functional form: log x_t = B_0 + B_1 * t + B_2 * t^2 + B_3 * i_t + B_4 * temp + B_5 * s_t-1 + lockdown + e_t

model_0 = ols('Load ~ t + t_2 + i_t + Temp_mean + Ema_90_1 + Lockdown_index', data=data).fit()
print(model_0.summary())

                            OLS Regression Results                            
Dep. Variable:                   Load   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     1462.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        17:05:55   Log-Likelihood:                 1836.8
No. Observations:                1244   AIC:                            -3660.
Df Residuals:                    1237   BIC:                            -3624.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -2.9376      0.295     -9.

In [331]:
# From previous step, the time index is not significant, drop

# Functional form: log x_t = B_0 + B_3 * i_t + B_4 * temp + B_5 * s_t-1 + lockdown + e_t

model_1 = ols('Load ~ i_t + Temp_mean + Ema_90_1 + Lockdown_index', data=data).fit()
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   Load   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     2160.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        17:05:55   Log-Likelihood:                 1827.7
No. Observations:                1244   AIC:                            -3645.
Df Residuals:                    1239   BIC:                            -3620.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -3.0258      0.297    -10.

In [332]:
# Enhance model_1 with piecewise temperature link
# We previously used a linear model for temperature, let's try the piecewise linear model

model_2 = ols('Load ~ i_t + T_mean_15_lt + T_mean_15_bt +  Ema_90_1 + Lockdown_index', data=data).fit()
print(model_2.summary())
data['in_forecast_2'] = model_2.predict(data)

                            OLS Regression Results                            
Dep. Variable:                   Load   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     1801.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        17:05:56   Log-Likelihood:                 1850.7
No. Observations:                1244   AIC:                            -3689.
Df Residuals:                    1238   BIC:                            -3659.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -2.8998      0.292     -9.

In [333]:
# Show in-sample forecasts
cutoff = dt.datetime(2018,1,1)
plt_df = data[data['Date']>=cutoff]
fig = go.Figure()
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Load'],
                    mode='lines',
                    name='Log load'))
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['in_forecast_2'],
                    mode='lines',
                    name='Forecast M2'))
fig.show()

In [334]:
# Scatter of observed vs forecast
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Load'], y=data['in_forecast_2'],
                    mode='markers',
                    text=data['Date'],
                    marker=dict(size=5,
                                opacity=0.5),
                    marker_color=data['Weekend'],
                    name='Log load'))
fig.add_trace(go.Scatter(x=data['Load'], y=data['Load'],
                    mode='markers',
                    marker=dict(size=2,
                                color='Gray'),
                    name='y = x'))

fig.update_layout(
    title="In-sample forecast vs observed",
    xaxis_title="Observed log load",
    yaxis_title="Forecast log load"
)

fig.show()

In [335]:
# Forecast one point
model_2.predict({'i_t':i_factors[i_factors['Day']==0]['i_t'], 'T_mean_15_lt':0, 'T_mean_15_bt':20, 'Ema_90_1':10, 'Lockdown_index':66.67})[0]

9.899004887434435

In [336]:
np.std(data['in_forecast_2']-data['Load'])

0.05465800120615977

In [337]:
# Inspect errors
# There seems to be some seasonality here - we're too high in Apr-May
# Could it be that daily seasonality is not constant over the months?
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Date'], y=data['in_forecast_2']-data['Load'],
                    mode='lines',
                    name='Log load'))
fig.show()

In [338]:
# Revisiting seasonality factors: want to consider day & month
# We calculate seasonality factors for each day of the week by month (assuming the factors are the same over the years)

avg = data[data['Date']<dt.datetime(2020,3,20)]['Ema_90'].mean()

i_new = data[data['Date']<dt.datetime(2020,3,20)][['Ema_90','Month','Day']].groupby(['Month','Day']).mean().reset_index()
i_new.rename(columns={'Ema_90':'Avg_Ema_90_monthly'},inplace=True)
i_new['i_new'] = i_new['Avg_Ema_90_monthly'] / avg

# Map this back to the df
data = data.merge(i_new[['Day','Month','i_new']], on=['Day','Month'],how='left')
data.head()

Date  Temp_min  Temp_mean  ...  T_mean_ma_15_bt  in_forecast_2     i_new
0 2017-01-01       4.4        5.2  ...              0.0            NaN  1.007547
1 2017-01-02      -0.6        2.0  ...              0.0      10.330686  1.016263
2 2017-01-03      -1.2        2.7  ...              0.0      10.384515  1.018303
3 2017-01-04       2.2        4.9  ...              0.0      10.419012  1.018169
4 2017-01-05      -3.2        1.2  ...              0.0      10.481549  1.020680

[5 rows x 36 columns]

In [339]:
# Model M5
# New seasonality factors, include interaction term for hot weekends, holidays and a general factor for lockdown
# The assumption here is that lockdown depresses load however we don't have enough info to specify a better functional form
# Could recompute the seasonality factors accounting for lockdown and not lockdown but not enough data to be able to generalise

model_5 = ols('Load ~ i_new + T_mean_15_lt + T_mean_15_bt +  Ema_90_1 + T_mean_15_bt * Weekend + Holiday + Lockdown_index', data=data).fit()
print(model_5.summary())
data['in_forecast_5'] = model_5.predict(data)


                            OLS Regression Results                            
Dep. Variable:                   Load   R-squared:                       0.928
Model:                            OLS   Adj. R-squared:                  0.928
Method:                 Least Squares   F-statistic:                     1997.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        17:05:56   Log-Likelihood:                 2174.9
No. Observations:                1244   AIC:                            -4332.
Df Residuals:                    1235   BIC:                            -4286.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                3.1739 

In [340]:
# Model M6
# Add a moving average for temperature, the idea being there could be patches of fair or cold weather

model_6 = ols('Load ~ i_new + T_mean_15_lt + T_mean_15_bt +  Ema_90_1 + T_mean_15_bt * Weekend + Holiday + Lockdown_index + T_mean_ma_15_lt + T_mean_ma_15_bt ', data=data).fit()
print(model_6.summary())
data['in_forecast_6'] = model_6.predict(data)

                            OLS Regression Results                            
Dep. Variable:                   Load   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                     1648.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        17:05:56   Log-Likelihood:                 2193.8
No. Observations:                1244   AIC:                            -4366.
Df Residuals:                    1233   BIC:                            -4309.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                3.8041 

In [341]:
# Show in-sample forecasts
cutoff = dt.datetime(2020,1,1)
plt_df = data[data['Date']>=cutoff]
fig = go.Figure()
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Load'],
                    mode='lines',
                    name='Log load'))

fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['in_forecast_5'],
                    mode='lines',
                    name='Forecast_M5'))
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['in_forecast_6'],
                    mode='lines',
                    name='Forecast_M6'))
fig.show()

# Scatter of observed vs forecast
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Load'], y=data['in_forecast_5'],
                    mode='markers',
                    text=data['Date'],
                    marker=dict(size=5,
                                opacity=0.5),
                    marker_color=data['Weekend'],
                    name='Log load'))
fig.add_trace(go.Scatter(x=data['Load'], y=data['Load'],
                    mode='markers',
                    marker=dict(size=2,
                                color='Gray'),
                    name='y = x'))

fig.update_layout(
    title="In-sample forecast vs observed",
    xaxis_title="Observed log load",
    yaxis_title="Forecast log load"
)

fig.show()

# Inspect errors
fig = go.Figure()

fig.add_trace(go.Scatter(x=data['Date'], y=data['in_forecast_5']-data['Load'],
                    mode='lines',
                    name='Error Model 5'))

fig.add_trace(go.Scatter(x=data['Date'], y=data['in_forecast_6']-data['Load'],
                    mode='lines',
                    name='Error Model 6'))
fig.show()

In [342]:
# For prediction

### Update the variables below ###

day = 3 # [Monday = 1, Sunday = 6]
month = 5 # May =5
temp = (16.0 + 18.0 + 18.0)/3 # TODO: We need to figure out how to go from London, Bristol and Leeds weather forecast to one overall forecast
prev_ema = 9.900249 # Look this up from dataframe
is_wkend = 0 # 0 = weekday, 1 = weekend
is_holiday = 0 # 0 = not, 1 = bank holiday
is_lockdown = 66.67 # this is a percent e.g. 66.67

### Leave this stuff ###

if temp > 15.5:
  temp_2 = temp
  temp_1 = 0
else:
  temp_2 = 0
  temp_1 = temp

i_new_val = float(i_new[(i_new['Day']==day)&(i_new['Month']==month)]['i_new'])

print('The forecast is: '+ str(model_5.predict(pd.DataFrame({'i_new':i_new_val,'T_mean_15_lt':temp_1,'T_mean_15_bt':temp_2, 
                                                             'Ema_90_1':prev_ema, 'Weekend':is_wkend,'Holiday':is_holiday,
                                                             'Lockdown_index':is_lockdown},index=[0]))[0]))

# For the error, possibly need to split the regression into train and test set and work out the error on the test set and report that
# Not clear how the test set works as technically we are using some exponentially weighted average in the calculation which sees info from all previous steps
# For now using the overall standard deviation

print('Error is: ' + str(np.std(data['in_forecast_5']-data['Load'])))

The forecast is: 9.942052939384451
Error is: 0.0421184427309292


In [0]:
# Using the data provided as of lunchtime Wednesday 27 (load up to 26 May and weather forecast up to 28 May)
# Need to forecast load for N+1 i.e. 28 May
# Step 1: use the model to forecast load for 27 May, using temperature forecast for 27 May (average of mean of 3 cities); forecast is 9.889952197763487, error 0.043
# Step 2: put the forecast for 27 May into 'Load' for 27 May and re-run (using weather forecast for 28 May, average of 3 city mean)
# Forecast is 9.925255, error: 0.04299
# Submit: Forecast = 9.93, error = 0.04

In [347]:
# Tracking the impact of lockdown_index

# Create forecasts with Lockdown_index = 0
data_copy = data.copy()
data_copy = data_copy.iloc[:-2]
data_copy['Lockdown_index'] = 0
data_copy['M6_forecast_no_lock'] = model_6.predict(data_copy)

# Plot series
cutoff = dt.datetime(2019,11,1)
plt_df = data_copy[data['Date']>=cutoff]
fig = go.Figure()
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['Load'],
                    mode='lines',
                    name='Log load'))

fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['in_forecast_6'],
                    mode='lines',
                    name='Forecast_M6'))
fig.add_trace(go.Scatter(x=plt_df['Date'], y=plt_df['M6_forecast_no_lock'],
                    mode='lines',
                    name='Forecast_M6_no_lock'))
fig.update_layout(legend_orientation='h')
fig.update_layout(
    title="Electricity load forecasting",
    xaxis_title="Date",
    yaxis_title="Log(load in MW)",
    title_x=0.5
)
fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

